<a href="https://colab.research.google.com/github/andrkech/GENERATIVE-METHODS-IN-GENOMICS-v2/blob/main/Copy_of_PHRED_TRANSFORMER_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [47]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import time

### GPU

In [2]:
DEVICE_NAME = tf.test.gpu_device_name()
if DEVICE_NAME != '/device:GPU:0':
    print('GPU device not found. Training on CPU.')
else:
    print('Found GPU at: {}'.format(DEVICE_NAME))

Found GPU at: /device:GPU:0


### Drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Dataset.

In [32]:
data_path = "/content/drive/MyDrive/BIOINFORMATICS/THESIS_KECHAGIAS/DATA/DATASET/phred300_tf_dataset"

element_spec = tf.TensorSpec(shape=(300, 1), dtype=tf.float32)
tf_dataset = tf.data.experimental.load(data_path, element_spec=element_spec)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


In [33]:
type(tf_dataset)

tensorflow.python.data.ops.load_op._LoadDataset

In [36]:
iterator = iter(tf_dataset)
first_element = next(iterator)
print(first_element)

print(len(tf_dataset))

tf.Tensor(
[[0.3655914 ]
 [0.3655914 ]
 [0.3655914 ]
 [0.3655914 ]
 [0.3655914 ]
 [0.40860215]
 [0.40860215]
 [0.39784947]
 [0.3655914 ]
 [0.39784947]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.39784947]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.39784947]
 [0.40860215]
 [0.37634408]
 [0.39784947]
 [0.37634408]
 [0.39784947]
 [0.40860215]
 [0.39784947]
 [0.37634408]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.39784947]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.3655914 ]
 [0.39784947]
 [0.39784947]
 [0.39784947]
 [0.40860215]
 [0.40860215]
 [0.40860215]
 [0.3655914 ]
 [0.38709676]
 [0.11827957]
 [0.3655914 ]
 [0.38709676]
 [0.3655914 ]
 [0.37634408]
 [0.39784947]
 [0.40860215]
 [0.40860215]
 [0.39784947]
 [0.40860215]
 [0.33333334]
 [0.39784947]
 [0.40860215]
 [0.37634408]
 [0.39784

In [7]:
def plot_quality_distributions(tf_dataset):
    for batch, qualities_batch in enumerate(tf_dataset):
        for i in range(qualities_batch.shape[0]):
            plt.hist(qualities_batch[i,:,0], bins=20)
            plt.title(f"Quality Score Distribution for Sequence {i} in Batch {batch}")
            plt.xlabel("Quality Score")
            plt.ylabel("Frequency")
            plt.show()

#plot_quality_distributions(dataset)

Hyperparameters

In [21]:
# Define model hyperparameters
num_layers = 4  # Number of encoder layers
d_model = 128  # Dimensionality of the model  <-- Define d_model here
num_heads = 8  # Number of attention heads
dff = 512  # Dimensionality of the feed-forward network
input_vocab_size = 1000  # Size of the vocabulary (adjust if needed)
maximum_position_encoding = 10000  # Maximum sequence length (adjust as needed)
dropout_rate = 0.1

BUild the model.

In [45]:
def positional_encoding(position, d_model):
    '''
    Positional Encoding

    Args:
        position: The max position in the sequence
        d_model: The dimensionality of a model

    Returns:
        The positional encoding vector
    '''
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1/ np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [48]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads, trainable=True)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2  =tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(outl1)
        ffn_output = seld.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

'''
Code heavily based on:
https://www.analyticsvidhya.com/blog/2021/01/implementation-of-attention-mechanism-for-caption-generation-on-transformers-using-tensorflow/
https://ithelp.ithome.com.tw/articles/10271277?sc=iThomeR
'''

'\nCode heavily based on:\nhttps://www.analyticsvidhya.com/blog/2021/01/implementation-of-attention-mechanism-for-caption-generation-on-transformers-using-tensorflow/\nhttps://ithelp.ithome.com.tw/articles/10271277?sc=iThomeR\n'

In [12]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __inti__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)

        return output, attention_weights

In [13]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [16]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [18]:
class TransformerEncoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 maximum_position_encoding, rate=0.1):
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x

In [26]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  # Cast step to float32
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [27]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [29]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

In [40]:
# Calculate the number of samples to take for training
train_size = int(len(tf_dataset) * 0.8)

# Split the dataset
train_dataset = tf_dataset.take(train_size)
val_dataset = tf_dataset.skip(train_size)

# Batch the datasets
training_dataset = train_dataset.batch(BATCH_SIZE)
validation_dataset = val_dataset.batch(BATCH_SIZE)

# Example: Print the first batch from the training dataset
for (batch, inp) in enumerate(training_dataset):
    print(f"Batch {batch}: {inp}")
    break  # Print only the first batch

Batch 0: [[[0.3655914 ]
  [0.3655914 ]
  [0.3655914 ]
  ...
  [0.32258064]
  [0.16129032]
  [0.29032257]]

 [[0.3655914 ]
  [0.3655914 ]
  [0.3655914 ]
  ...
  [0.11827957]
  [0.17204301]
  [0.23655914]]

 [[0.34408602]
  [0.12903225]
  [0.12903225]
  ...
  [0.19354838]
  [0.07526882]
  [0.07526882]]

 ...

 [[0.3655914 ]
  [0.3655914 ]
  [0.29032257]
  ...
  [0.09677419]
  [0.07526882]
  [0.11827957]]

 [[0.3655914 ]
  [0.3655914 ]
  [0.3655914 ]
  ...
  [0.40860215]
  [0.17204301]
  [0.19354838]]

 [[0.3548387 ]
  [0.3655914 ]
  [0.3548387 ]
  ...
  [0.29032257]
  [0.2795699 ]
  [0.23655914]]]


In [41]:
def train_transformer(transformer, train_dataset, val_dataset, num_epochs, log_dir):
    """Trains the Transformer model."""

    # Define metrics
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy')

    # Create checkpoint object and manager
    checkpoint = tf.train.Checkpoint(transformer=transformer,
                                    optimizer=optimizer)
    checkpoint_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)

    # Create TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Training loop
    for epoch in range(num_epochs):
        start = time.time()

        train_loss.reset_states()
        train_accuracy.reset_states()

        # Iterate over the training dataset
        for (batch, inp) in enumerate(train_dataset):
            # Prepare target sequence (shifted by one position)
            tar = inp[:, 1:]
            inp = inp[:, :-1]

            train_step(inp, tar)  # Call the train_step function

            if batch % 50 == 0:
                print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

        # Optional: Evaluate on validation set and log metrics to TensorBoard
        # ...

        print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
        print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

In [42]:
def create_masks(inp, tar):
    """Creates all masks for training."""

    # Encoder padding mask (Used in the 1st attention block in the encoder)
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

def create_padding_mask(seq):
    """Creates a padding mask for a sequence."""

    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    """Creates a look-ahead mask for a sequence."""

    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [49]:
transformer = TransformerEncoder(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=dropout_rate)

TypeError: Layer.__init__() got multiple values for argument 'trainable'